# ⚒️ Tool calling via chat model (OpenAI)

- Table of contents
  - [Tool calling via chat model](#️-tool-calling-via-chat-model-openai)
  - [Tool output to Chat Mode](#️-pass-tool-output-to-chat-models)
  - [Pass Tool value in run time](#how-to-pass-run-time-value-to-the-tool)
  - [How to stream tool calls](#how-to-stream-tool-calls)

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.tools import tool


class InputSchema(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


@tool(args_schema=InputSchema)
def multiply(a: int, b: int):
    """Multiply two numbers"""

    return a * b


@tool(args_schema=InputSchema)
def add(a: int, b: int):
    """Add two numbers"""

    return a + b

10

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

tools = [multiply, add]
tool_by_name = {tool.name: tool for tool in tools}
llm_with_tools = llm.bind_tools(tools)

{'multiply': StructuredTool(name='multiply', description='Multiply two numbers', args_schema=<class '__main__.InputSchema'>, func=<function multiply at 0x00000200C75E99E0>), 'add': StructuredTool(name='add', description='Add two numbers', args_schema=<class '__main__.InputSchema'>, func=<function add at 0x00000200C75E9F80>)}


In [7]:
query = "what is 2 multiply by 5. then add by 5"

print(llm_with_tools.invoke(query).tool_calls)

[{'name': 'multiply', 'args': {'a': 2, 'b': 5}, 'id': 'call_2FRwjIPOQSZbB5y9JIWsERWp', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 5, 'b': 5}, 'id': 'call_UTla1eAWzA383cldj47biA4z', 'type': 'tool_call'}]


## ⚒️ Pass tool output to chat models

![pass-tool-output-to-chat-models](https://python.langchain.com/assets/images/tool_invocation-7f277888701ee431a17607f1a035c080.png)

![pass-tool-output-to-chat-models](https://python.langchain.com/assets/images/tool_results-71b4b90f33a56563c102d91e7821a993.png)

In [10]:
from langchain_core.messages import HumanMessage

query = "what is 3 * 2? Also, what is 11 + 49"

messages = [HumanMessage(content=query)]

ai_message = llm_with_tools.invoke(messages)

print(ai_message)

messages.append(ai_message)

content='' additional_kwargs={'tool_calls': [{'id': 'call_n7dPtZmrw7IsD0aShBwKRhRH', 'function': {'arguments': '{"a": 3, "b": 2}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_WtoOMhOAwKdvfga0jMFeyncd', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 96, 'total_tokens': 147, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-37d38044-dda7-44af-8362-1da3fabf2fd7-0' tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_n7dPtZmrw7IsD0aShBwKRhRH', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_WtoOMhOAwKdvfga0jMFeyncd

In [18]:
for tool_call in ai_message.tool_calls:
    select_tool = tool_by_name[tool_call["name"]]
    tool_msg = select_tool.invoke(tool_call)

    messages.append(tool_msg)

print(messages)

[HumanMessage(content='what is 3 * 2? Also, what is 11 + 49', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_n7dPtZmrw7IsD0aShBwKRhRH', 'function': {'arguments': '{"a": 3, "b": 2}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_WtoOMhOAwKdvfga0jMFeyncd', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 96, 'total_tokens': 147, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-37d38044-dda7-44af-8362-1da3fabf2fd7-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_n7dPtZmr

In [19]:
llm_with_tools.invoke(messages)

AIMessage(content='The result of \\(3 \\times 2\\) is \\(6\\), and \\(11 + 49\\) equals \\(60\\).', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 162, 'total_tokens': 193, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'finish_reason': 'stop', 'logprobs': None}, id='run-831e1509-dc9b-4ebf-bfaf-f9d825281c2c-0', usage_metadata={'input_tokens': 162, 'output_tokens': 31, 'total_tokens': 193, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## How to pass run time value to the tool

- You may need to bind value to a tool that are only know at runtime.
- The tool may required the user id who invoke the tool.
- We can't pass the tool to LLM's for security risk instead of LLM's may control parameters.
- We pass the `user_id` during invoke of the tool by using `InjectedToolArg` annotation.
- `InjectedToolArg` is used to mark annotation on some parameters, `user_id` as begin injected at runtime meaning they should not being generated by the model. 

In [79]:
from typing import List

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool, InjectedToolArg
from typing_extensions import Annotated

llm = ChatOpenAI(model="gpt-4o-mini")

In [143]:
user_to_pets = {}


@tool(parse_docstring=True)
def update_favorite_pet(pets: List[str], user_id: Annotated[str, InjectedToolArg]) -> None:
    """
    Add the list of favorite pets.

    Args:
        pets: The list of favorite pets to set.
        user_id: User's Id
    """
    user_to_pets[user_id] = pets


@tool(parse_docstring=True)
def delete_favorite_pet(user_id: Annotated[str, InjectedToolArg]) -> None:
    """
    Delete the list of favorite pets

    Args:
        user_id: User's Id
    """
    if user_id in user_to_pets:
        del user_to_pets[user_id]


@tool(parse_docstring=True)
def list_favorite_pet(user_id: Annotated[str, InjectedToolArg]) -> None:
    """
    List favorite pets of any.

    Args:
        user_id: User's Id
    """
    return user_to_pets.get(user_id, [])

- If we look input schemas for these tool, we'll see that `user_id` is still listed

In [10]:
update_favorite_pet.get_input_schema().model_json_schema()

{'description': 'Add the list of favorite pets.',
 'properties': {'pets': {'description': 'The list of favorite pets to set.',
   'items': {'type': 'string'},
   'title': 'Pets',
   'type': 'array'},
  'user_id': {'description': "User's Id",
   'title': 'User Id',
   'type': 'string'}},
 'required': ['pets', 'user_id'],
 'title': 'update_favorite_pet',
 'type': 'object'}

- If we look at that passed into the model, we see there is not `user_id`. 

In [11]:
update_favorite_pet.tool_call_schema.model_json_schema()

{'description': 'Add the list of favorite pets.',
 'properties': {'pets': {'description': 'The list of favorite pets to set.',
   'items': {'type': 'string'},
   'title': 'Pets',
   'type': 'array'}},
 'required': ['pets'],
 'title': 'update_favorite_pet',
 'type': 'object'}

- So let's invoke our tools.

In [81]:
user_id = "123"

update_favorite_pet.invoke(
    {
        "pets": ["pandas", "koala"],
        "user_id": user_id
    }
)

list_favorite_pet.invoke({"user_id": user_id})

['pandas', 'koala']

In [144]:
tools = [
    update_favorite_pet,
    delete_favorite_pet,
    list_favorite_pet
]

tool_by_name = {tool.name: tool for tool in tools}
llm_with_tools = llm.bind_tools(tools)

In [23]:
ai_msg = llm_with_tools.invoke("my favorite animals are cats and parrots")
ai_msg.tool_calls

[{'name': 'update_favorite_pet',
  'args': {'pets': ['cats', 'parrots']},
  'id': 'call_yuyLzfrAWY5n9g2wJRzssJt4',
  'type': 'tool_call'}]

- See there is not `user_id` field in the tool call, this is because of `InjectToolCall`.

- If we want to actually execute our tool using the model-generator we'll need to manually inject the `user_id`

In [ ]:
from copy import deepcopy
from langchain_core.runnables import chain


@chain
def injected_user_id(ai_msg):
    tool_calls = []
    for tool_call in ai_msg.tool_calls:
        tool_call_copy = deepcopy(tool_call)
        tool_call_copy["args"]["user_id"] = user_id
        tool_calls.append(tool_call_copy)

    return tool_calls


injected_user_id.invoke(ai_msg)

In [146]:
@chain
def tool_router(tool_call):
    return tool_by_name[tool_call["name"]]


chain = llm_with_tools | injected_user_id | tool_router.map()

In [149]:
chain.invoke("my favorite animals are pandas, parrots, koalas, cats, dogs")

[ToolMessage(content='null', name='update_favorite_pet', tool_call_id='call_4Yms1sD6Ahq0H0e5yDz5GDKw')]

In [150]:
user_to_pets

{'123': ['pandas', 'parrots', 'koalas', 'cats', 'dogs']}

## How to stream tool calls

- When tool are called in a streaming way, the message chunks will be populate with the tool call chunk object in a list via a the .tool_call_chunks attributes.
- A `ToolCallChunks` includes optional string fields such as `id`, `name`, `args` and includes an optional integer field index that can be used to join chunks together.
- Fields are optional because portions of a tool call may be streamed across different chunks.
- An AIMessageChunk with tool call chunks will also include .tool_calls and .invalid_tool_calls fields. 

In [1]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int):
    """Add two numbers."""
    return a + b


@tool
def multiply(a: int, b: int):
    """Multiply two numbers."""
    return a * b


tools = [add, multiply]

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

In [3]:
query = "what is 3 * 12, Also, what is 12 + 12"

async for chunk in llm_with_tools.astream(query):
    print(chunk.tool_call_chunks)

[]
[{'name': 'multiply', 'args': '', 'id': 'call_1vz0ngvV2B8Qun3sp2fer2VP', 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': '{"a"', 'id': None, 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': ': 3, ', 'id': None, 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': '"b": 1', 'id': None, 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': '2}', 'id': None, 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': 'add', 'args': '', 'id': 'call_5kuRsyqAgBtsBUnXYsHemqjP', 'index': 1, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': '{"a"', 'id': None, 'index': 1, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': ': 12,', 'id': None, 'index': 1, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': ' "b": ', 'id': None, 'index': 1, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': '12}', 'id': None, 'index': 1, 'type': 'tool_call_chunk'}]
[]


- The given message chunks will be merge their corresponding tool call chunks.

In [4]:
first = True

async for chunk in llm_with_tools.astream(query):
    if first:
        gathered = chunk
        first = False
    else:
        gathered = gathered + chunk

    print(gathered)

content='' additional_kwargs={} response_metadata={} id='run-4b64d5ad-5951-4fe8-8fee-9aa6f04006d0'
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_BRODoggbnSBckl8lwwEGUReF', 'function': {'arguments': '', 'name': 'multiply'}, 'type': 'function'}]} response_metadata={} id='run-4b64d5ad-5951-4fe8-8fee-9aa6f04006d0' tool_calls=[{'name': 'multiply', 'args': {}, 'id': 'call_BRODoggbnSBckl8lwwEGUReF', 'type': 'tool_call'}] tool_call_chunks=[{'name': 'multiply', 'args': '', 'id': 'call_BRODoggbnSBckl8lwwEGUReF', 'index': 0, 'type': 'tool_call_chunk'}]
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_BRODoggbnSBckl8lwwEGUReF', 'function': {'arguments': '{"a"', 'name': 'multiply'}, 'type': 'function'}]} response_metadata={} id='run-4b64d5ad-5951-4fe8-8fee-9aa6f04006d0' tool_calls=[{'name': 'multiply', 'args': {}, 'id': 'call_BRODoggbnSBckl8lwwEGUReF', 'type': 'tool_call'}] tool_call_chunks=[{'name': 'multiply', 'args': '{"a"', 'id': 'call_BRODoggbnSBckl